<div style="display: flex; align-items: center;">
  <img src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/SemanticKernelLogo.png" width="60px" style="margin-right: 10px;">
  <span style="font-size: 1.5em; font-weight: bold;">Semantic Kernel - Premortem (What-If) Decision Analysis  </span>
</div>

> 📜 **_"The pre-mortem method has a number of benefits besides identifying trouble spots in a plan – reducing overconfidence and promoting discoveries. The method can strengthen members’ mental models… And it can help create a culture of candor and trust."_**
>
> -- <cite>Gary Klein (Psychologist, creator of the Pre-mortem method)</cite>  

Decision Intelligence applied in this module:  
* Using Premortem Decision Analysis to improve decision quality & outcomes  
* Using a Premortem decision output for improving of Gathering Intelligence 
* Attain different perspectives using mutliple decision frames  
* Apply a Decision Framework to Premortem Decision Analysis 


### What is a Premortem (What-If Analysis)  

You have probably heard of a **postmortem (post-mortem)**, which is a look back at an actual failure (or success) to understand why it happened and learn from it. In the context of decision making, after the decision outcome is clear you can analyze why the result occured. For example, a hedge fund may host a postmortem meeting after analyzing an unsuccesful move on a tech stock that didn't materialize the return on investment as was planned. 

In a **premortem (pre-mortem)** the outcome is "imagined" and then it is figured out why that potential "imagined" outcome happened. In a premortem, a planning exercise is held where a team imagines that a project or decision has already succeeded or failed. The team then works backward, brainstorming what might have caused things to go wrong. This approach allows everyone to spot potential risks and address them before starting the project. Therefore, it’s a bit like predicting the future and using that prediction to make the forecast stronger. 

### Example Decision Scenario - Purchasing a Car   

Imagine you're about to buy a new car. Let's play a game of "What-If" that the purchase decision results in either two outcomes a: **disaster** or a **successful** outcome. 

#### Premortem Outcome - Disaster  

<img style="display: block; margin: auto;" width ="700px" src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/Scenarios/Scenario-DecisionPremortem.png">

In a **premortem (pre-mortem)**, you pretend the car purchase turned out to be a disaster. Now, you ask yourself:  
**What went wrong with the decision to buy a new car?**

Here are some potential reasons the car purchase had a poor outcome:  

* **Financial issues:** You realize later you can’t comfortably afford the monthly payments or maintenance costs.  
* **Unexpected repairs:** The car has hidden mechanical problems, leading to costly repairs.  
* **Insurance costs:** Insurance is much higher than expected, impacting your budget.
* **Poor fuel efficiency:** The car uses more gas than you thought, making it expensive to drive.
Resale value drop: The car’s value plummets quickly, leaving you with less if you want to sell or trade it.  

#### Premortem Outcome - Success

In a premortem for a successful car purchase, imagine you bought the car, and it turned out to be a succssful decision. Now, you ask yourself:  
**What went right with the decision to buy a new car?**  

Here are some potential reasons the car purchase had a good outcome: 

* **Affordable and realistic budget:** You set a budget that allowed for comfortable monthly payments and covered insurance, fuel, and maintenance without straining your finances.  
* **Thorough inspection:** You got a detailed inspection before buying, which confirmed the car was in excellent condition and minimized the risk of future repairs.
* **Reliable model choice:** You researched models with a reputation for reliability, so you experience fewer repairs and smooth performance.
* **Fuel efficiency matches needs:** You chose a car that meets your fuel efficiency needs, saving money on gas and fitting well with your daily driving habits.   
* **High resale value:** You picked a make and model that holds its value well, giving you confidence that it’s a good long-term investment.  

#### Premortem (What-If) Decision Analysis 

Imagining successful outcomes helps you focus on actions like sticking to a car budget, getting a full inspection, and choosing a reliable model. You will be more likely to make a purchase you’ll be happy with. By plyaing a "game" of What-If with decision criteria in advance, you could adapt your strategy. For example, you could consider to thoroughly check maintenance records, buy additional warranty, test drive with similar terrain in your area, or reassess your budget to avoid these potential problems.

How can Generative AI help? Using Generative AI, the LLM models can be used to "imagine" premortem outcomes and  brainstorm potential factors why that certain outcome occured. Generative AI agents can act as your "premortem" team. This can lead to additional questions, gathering of intelligence, checklists, negotiation leverage etc. you didn't consider initially. 

---
### Step 1 - Initialize Configuration Builder & Build the Semantic Kernel Orchestration 

In [21]:
#r "nuget: Microsoft.Extensions.Configuration, 9.0.6"
#r "nuget: Microsoft.Extensions.Configuration.Json, 9.0.6"
#r "nuget: Microsoft.SemanticKernel, 1.60.0"
#r "nuget: Microsoft.SemanticKernel.Plugins.Core, 1.60.0-preview"

using Microsoft.Extensions.Configuration.Json;
using Microsoft.Extensions.Configuration;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using System.IO;

var configurationBuilder = new ConfigurationBuilder()
    .SetBasePath(Directory.GetCurrentDirectory())
    .AddJsonFile("local.settings.json", optional: true, reloadOnChange: true)
    .AddJsonFile("secrets.settings.json", optional: true, reloadOnChange: true);
var config = configurationBuilder.Build();

// IMPORTANT: You ONLY NEED either Azure OpenAI or OpenAI connectiopn info, not both.
// Azure OpenAI Connection Info using o1 series of models
var azureOpenAIEndpoint = config["AzureOpenAI:Endpoint"];
var azureOpenAIAPIKey = config["AzureOpenAI:APIKey"];
var azureOpenAIModelDeploymentName = config["AzureOpenAI:ModelDeploymentName"];
// OpenAI Connection Info 
var openAIAPIKey = config["OpenAI:APIKey"];
var openAIModelId = config["OpenAI:ModelId"];

Installed Packages Microsoft.Extensions.Configuration, 9.0.6 Microsoft.Extensions.Configuration.Json, 9.0.6 Microsoft.SemanticKernel, 1.60.0 Microsoft.SemanticKernel.Plugins.Core, 1.60.0-preview

In [23]:
using Microsoft.Extensions.DependencyInjection.Extensions;
using Microsoft.Extensions.DependencyInjection;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;

Kernel semanticKernel;

// Set the flag to use Azure OpenAI or OpenAI. False to use OpenAI, True to use Azure OpenAI
var useAzureOpenAI = true;

// Create a new Semantic Kernel instance
if (useAzureOpenAI)
{
    Console.WriteLine("Using Azure OpenAI Service");
    semanticKernel = Kernel.CreateBuilder()
        .AddAzureOpenAIChatCompletion(
            deploymentName: azureOpenAIModelDeploymentName,
            endpoint: azureOpenAIEndpoint,
            apiKey: azureOpenAIAPIKey)
        .Build();
}
else
{
    Console.WriteLine("Using OpenAI Service");
    semanticKernel = Kernel.CreateBuilder()
        .AddOpenAIChatCompletion(
            modelId: openAIModelId,
            apiKey: openAIAPIKey)
        .Build();
}

var chatCompletionService = semanticKernel.Services.GetRequiredService<IChatCompletionService>();

Using Azure OpenAI Service


----
### Step 2 - Premortem Risk Analysis with AI Reasoning Models

> 📜 **_"Success is a lousy teacher. It seduces smart people into thinking they can’t lose."_**
>
> -- <cite>Bill Gates (Co-Founder of Microsoft, Philantropist)</cite>  

In the decision scenario below, an AI model will perform a premortem (What-If) analysis on the decision whether to purchase or lease a car. The AI model will provide risks and items to consider for each of the decision options: purchasing a car or leasing a car. 

In [25]:
// Create a system prompt instruction to override the default system prompt
// Add the System Prompt (Persona) to behave like a decision intelligence assistant
var systemPromptForDecisions = """
You are a Decision Intelligence Assistant. 
Assist the user in exploring options, reasoning through decisions, problem-solving.
Apply systems thinking to various scenarios. Provide structured, logical, and comprehensive advice.

Output Format Instructions: 
When generating Markdown, do not use any headings higher than ###. 
Avoid # and ## headers. Use only ###, ####, or lower-level headings if necessary. 
All top-level section headers should start at ### or lower. 
Never use ---, ***, or ___ for horizontal lines. There should be no horizontal lines in the output.
For separation, use extra extra spacing. Do not any render horizontal lines.
Format the response using only a Markdown table. Only return a Markdown table. 
Do not enclose the table in triple backticks.
""";
var decisionPreMortemAnalysisPrompt = """
Perform an advanced premortem (What-If) analysis on the following decision scenario:
You are deciding to purchase a new car versus leasing a car.
""";

// Create a new chat history with pre-mortem analysis prompt
var chatHistoryPreMortemAnalysis = new ChatHistory();
chatHistoryPreMortemAnalysis.AddSystemMessage(systemPromptForDecisions);
chatHistoryPreMortemAnalysis.AddUserMessage(decisionPreMortemAnalysisPrompt);

// Create a new OpenAI prompt execution settings object
// Try different settings (Temperature, FrequencyPenalty etc) to see how they affect the quality of the generated text
var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    MaxTokens = 2500, 
    Temperature = 0.3, 
    TopP = 1.0, 
    FrequencyPenalty = 0.0, 
    PresencePenalty = 0.0
    };

// Include both the chat history and the prompt execution settings in the call to GetStreamingChatMessageContentsAsync
var decisionResponseChainOfThought = await chatCompletionService.GetChatMessageContentAsync(chatHistoryPreMortemAnalysis, openAIPromptExecutionSettings);
decisionResponseChainOfThought.Content.DisplayAs("text/markdown");

// Add the AI response to the chat history
chatHistoryPreMortemAnalysis.AddAssistantMessage(decisionResponseChainOfThought.Content);


| Step                         | Description                                                                                                                                                                                                                     | Purchase Car                                                                                      | Lease Car                                                                                         |
|------------------------------|-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|---------------------------------------------------------------------------------------------------|---------------------------------------------------------------------------------------------------|
| **Objective**                | Define the primary goal of the decision.                                                                                                                                                                                           | Own a vehicle, maximize long-term value, flexibility in usage.                                    | Access a new vehicle with lower upfront costs, flexibility to change cars frequently.             |
| **Key Assumptions**          | List assumptions that may impact the outcome.                                                                                                                                                                                      | Will keep the car for 5+ years; can afford higher upfront costs; maintenance responsibility.       | Will not exceed mileage limits; prefer new cars every 2-3 years; accept long-term payments.        |
| **Success Criteria**         | What does success look like?                                                                                                                                                                                                       | Reliable transportation, lower total cost over time, asset ownership, customization.               | Predictable monthly payments, minimal maintenance, ability to upgrade regularly.                   |
| **Potential Failure Modes**  | Identify ways the decision could go wrong.                                                                                                                                                                                         | High depreciation, unexpected repair costs, rapid technology change, lifestyle changes.            | Excess mileage fees, wear-and-tear charges, lack of equity, higher long-term cost.                 |
| **What-If Scenarios**        | Explore plausible negative scenarios for each option.                                                                                                                                                                              | - Major accident or breakdown early on.<br>- Need to relocate or change vehicle type soon.<br>- Resale value drops sharply.<br>- Maintenance costs spike unexpectedly. | - Exceed mileage allowance.<br>- Early lease termination needed.<br>- Life changes require different vehicle.<br>- Lease terms become unfavorable. |
| **Early Warning Signs**      | What indicators suggest the decision is going off-track?                                                                                                                                                                           | Frequent repairs, declining resale values, dissatisfaction with vehicle, financial strain.         | Approaching mileage limits, dissatisfaction with lease terms, unexpected fees, lifestyle changes.  |
| **Mitigation Strategies**    | Actions to prevent or reduce negative outcomes.                                                                                                                                                                                    | Purchase extended warranty, research resale values, maintain vehicle well, consider certified pre-owned. | Negotiate higher mileage allowance, understand lease terms, budget for fees, consider lease buyout options. |
| **Contingency Plans**        | Steps to take if a negative scenario occurs.                                                                                                                                                                                       | Sell or trade in vehicle, refinance loan, use public transport temporarily, set aside repair fund. | Transfer lease, negotiate early termination, switch to purchase if needed, use alternative transport. |
| **Long-Term Implications**   | Consider the impact over a longer time horizon.                                                                                                                                                                                    | Ownership after loan payoff, potential for lower total cost, asset value, more flexibility.        | Continuous payments, no ownership, frequent vehicle upgrades, possible higher cumulative cost.     |
| **Stakeholder Impact**       | How does the decision affect others (family, finances, environment)?                                                                                                                                                               | Family can use/sell car as needed, possible higher environmental impact if keeping older car.      | Family may have limited flexibility, newer cars may be more efficient, but less control over asset.|
| **Decision Triggers**        | What factors might prompt a re-evaluation of the decision?                                                                                                                                                                         | Major repair bills, job/life changes, financial hardship, new vehicle incentives.                  | Lease-end approaching, lifestyle changes, dissatisfaction with terms, better purchase incentives.   |

The large reasoning model provided a comprehensive set of criteria, risks, mitigation strategy and approaches for the two decisions. This is a great option, but it might be more ideal to provide this in an easy to consume table (checklist) for the user that is easy to compare and contrast. This What-If analysis is easy to consume on a mobile device or even print out and annotate. 

This Markdown table can be further simplified by highlighting which risk factor is more favorable when comparing a car purchase versus a car lease. This can make the decision process much easier to the user, as at a glance they can see overwhelming evidence that favors once decision option over the other.  

In [26]:
var simplifyRiskAnalysisPrompt = """
You are provided a detailed markdown table in the above chat history that 
compares Purchase and Lease options across various risk factors and uncertainties. 
Your task is to convert this table into a simplified markdown table that uses check marks (☑️) 
to indicate which option is less risky or more ideal for each risk factor. 
The output table should have three columns: "Risk Factor / Uncertainty / Potential Outcome", "Purchase", and "Lease". 
For each risk factor, decide based on the detailed descriptions which option (Purchase or Lease) 
is preferable (i.e., less risky or more advantageous) and place a check mark (☑️) in that column. 
If an option is not the ideal choice for a given risk factor, leave its cell blank. 
Make sure the simplified table closely reflects the key insights from the detailed table. 

Provide a small summary below the table of the key insights 
from the detailed table in a single paragraph before the simplified table.
""";

chatHistoryPreMortemAnalysis.AddUserMessage(simplifyRiskAnalysisPrompt);

var preMortemAnalysisSimplifiedChecklistResponse = await chatCompletionService.GetChatMessageContentAsync(chatHistoryPreMortemAnalysis, openAIPromptExecutionSettings);
preMortemAnalysisSimplifiedChecklistResponse.Content.DisplayAs("text/markdown");


The detailed analysis shows that purchasing a car is generally more advantageous for those seeking long-term value, ownership flexibility, and lower total cost over time, especially if they plan to keep the car for several years. Leasing, on the other hand, is preferable for those who prioritize lower upfront costs, predictable payments, access to newer vehicles, and minimal maintenance responsibilities, but it carries risks related to mileage limits, lease terms, and lack of ownership. Each option has unique risks and benefits depending on personal circumstances and priorities.

| Risk Factor / Uncertainty / Potential Outcome             | Purchase | Lease |
|----------------------------------------------------------|----------|-------|
| Lower long-term total cost                               | ☑️       |       |
| Flexibility in usage and ownership                       | ☑️       |       |
| Lower upfront costs                                      |          | ☑️    |
| Predictable monthly payments                             |          | ☑️    |
| Minimal maintenance responsibility                       |          | ☑️    |
| Ability to upgrade vehicles frequently                   |          | ☑️    |
| Avoiding excess mileage/wear-and-tear fees               | ☑️       |       |
| Asset ownership and resale value                         | ☑️       |       |
| Protection from rapid depreciation                       |          | ☑️    |
| Ability to customize vehicle                             | ☑️       |       |
| Lower risk if lifestyle changes or relocation needed     |          | ☑️    |
| Lower risk of financial strain from unexpected repairs   |          | ☑️    |
| Family flexibility and control over vehicle              | ☑️       |       |
| Lower cumulative cost if keeping car long-term           | ☑️       |       |
| Simpler exit if dissatisfied or needs change             | ☑️       |       |

### Step 3 - Premortem Analysis for Radio Telescope Operations 

Exploring space with telescopes is a time consuming investment. Current telescope designs are limited to exploring small areas of space in a certain time frame. Without going into details, the process is much more complex than snapping a picture with a mobile device. Telescopes can be pointed at a specific space area for days, weeks or even more than months! If a telescope is exploring a certain area of space it isn't collecting data on other areas. Scientists and researchers compete for valuable access to telescope resources by providing observatories what space area they want to explore, any cellestial contraints, timeframes, calibration costs, data collection transfer etc. This presents a very interesting Decision Intelligence constraints problem that observatories must manage. 

<img style="display: block; margin: auto;" width ="700px" src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/Scenarios/Scenario-RadioTelescope.png">

#### Decision Scenario - Premortem for Radio Telescope Operations 

> 📜 "We choose to go to the Moon in this decade and do the other things, not because they are easy, but because they are hard." 
>
> -- <cite>John F. Kennedy (35th President of the United States)</cite>  

**Intro to Decision Scenario:** A newly commissioned radio telescope stands on the verge of unveiling the proof of alien life, but it is still far from fully calibrated. Despite limited data processing capacity and a software suite that remains untested, an unexpected cosmic event has just begun, hinting at signals that could reveal alien life. Faced with the risk of hardware damage or data corruption from running the telescope prematurely, scientists debate whether to switch it on now or wait until everything is fully operational—knowing they could lose a once-in-a-generation opportunity. Determined not to squander this brief cosmic window, the team conducts a premortem analysis to predict what might go wrong. Armed with these insights, they can better prepare for potential pitfalls and strengthen their chances of catching historic signals from across the galaxy.

**Pre-Mortem Risk Scenario:** Let's assume (pretend) the scientists decide to rush the radio telescope to operate early to find alien life. Furthermore, assume a that this decision has led to a disaster with hardware failure to the telescope. Even though the (What-If) decision was made to rush the radio telescope operationally, formal scientific telescope operations have been followed. The goal of this premortem (What-If analysis) is to illuminate any risk mitigation strategies and craft a risk mitigation approach that may have been uncovered based on the premortem. 

Techinques like this a very powerful in a variety of business scenarios, where existing operational frameworks exist but simulating an outcome can uncover gaps, emerging risks and other opportunities to optimize. 

In [29]:
// Create a system prompt instruction to override the default system prompt
// Add the System Prompt (Persona) to behave like a decision intelligence assistant
var systemPromptForPremortemRadioTelescope = """
You are a Decision Intelligence Assistant. 
Assist the user in exploring options, reasoning through decisions, problem-solving.
Apply systems thinking to various scenarios. Provide structured, logical, and comprehensive advice.

Output Format Instructions: 
When generating Markdown, do not use any headings higher than ###. 
Avoid # and ## headers. Use only ###, ####, or lower-level headings if necessary. 
All top-level section headers should start at ### or lower. 
Never use ---, ***, or ___ for horizontal lines. There should be no horizontal lines in the output.
For separation, use extra extra spacing. Do not any render horizontal lines.
Format the response using only a Markdown table. Only return a Markdown table. 
Do not enclose the table in triple backticks.

Provide a quick summary of the pre-mortem analysis below the table of the key insights.
""";

var decisionAnalysisPromptForPremortemRadioTelescope = """
You are a project manager at a new radio telescope facility looking for alien life.  
It is still far from fully calibrated. Despite limited data processing capacity and a software suite that remains untested, 
an unexpected cosmic event has just begun, hinting at signals that could reveal alien life. 
Faced with the risk of hardware damage or data corruption from running the telescope prematurely, 
scientists debate whether to switch it on now or wait until everything is fully operational—knowing 
they could lose a once-in-a-generation opportunity. 

Imagine it is six months from now and the project to detect potential alien signals from the sudden cosmic event has failed. 
The radio telescope was likely activated prematurely, software remained untested, and the team rushed to capture transmissions. 
Despite the urgency, the outcome was disastrous: hardware suffered damage under the increased load, 
data came out corrupted, or no meaningful signals were confirmed. 

Now, step into that moment of failure—what specific issues, oversights, or cascading errors caused this grim scenario?

1) Envision every possible way the radio telescope's early deployment could have led to compromised or unusable data.
2) Identify how incomplete software testing and rushed calibration might have 
contributed to errors in processing or storing signals. 
3) Consider any overlooked safety protocols or hardware precautions that might have led to damage or malfunctions.
4) Examine the roles and responsibilities of the team and think about communication breakdowns, 
rushed decisions, and missed signals or deadlines.
5) Assess how time pressure from the rare cosmic event might have amplified smaller mistakes and 
turned them into systemic failures.

Use the outcomes of this “what if” vision to create a detailed list of preventive measures. 
Then, propose fail-safes, additional resources, or procedural changes to ensure that, 
should you choose to activate the telescope early, you can still capture the event safely and 
effectively—maximizing the chance to detect historic signals without jeopardizing the entire operation.
""";

// Create a new chat history with pre-mortem analysis prompts
var chatHistoryPreMortemTelescopeAnalysis = new ChatHistory();
chatHistoryPreMortemTelescopeAnalysis.AddSystemMessage(systemPromptForPremortemRadioTelescope);
chatHistoryPreMortemTelescopeAnalysis.AddUserMessage(decisionAnalysisPromptForPremortemRadioTelescope);

// Create a new OpenAI prompt execution settings object
// Try different settings (Temperature, FrequencyPenalty etc) to see how they affect the quality of the generated text
var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    MaxTokens = 8000, 
    Temperature = 0.3, 
    TopP = 1.0, 
    FrequencyPenalty = 0.0, 
    PresencePenalty = 0.0
    };

var preMortemTelescopeAnalysisResponse = await chatCompletionService.GetChatMessageContentAsync(chatHistoryPreMortemTelescopeAnalysis, openAIPromptExecutionSettings);
preMortemTelescopeAnalysisResponse.Content.DisplayAs("text/markdown");

| Issue/Failure Mode                                                                 | Contributing Factors                                                                                  | Preventive Measures                                                                                         | Proposed Fail-safes/Resources/Procedural Changes                                                      |
|------------------------------------------------------------------------------------|------------------------------------------------------------------------------------------------------|-------------------------------------------------------------------------------------------------------------|-------------------------------------------------------------------------------------------------------|
| Data corruption due to untested software pipelines                                 | Incomplete software testing, unknown bugs in data handling, unverified storage protocols              | Conduct rapid but targeted software validation; run simulated data through pipelines before live activation | Implement real-time data integrity checks; maintain raw data backups; allocate on-call software support|
| Hardware damage from overload or improper configuration                            | Calibration incomplete, hardware limits not well understood, safety interlocks bypassed               | Set conservative operational parameters; review hardware safety thresholds                                 | Install automated shutdown triggers; assign hardware monitoring team; keep spare parts on hand         |
| Loss of critical data due to storage bottlenecks or misconfiguration               | Limited processing/storage capacity, rushed setup, no redundancy                                      | Prioritize essential data streams; pre-configure storage for high-throughput                               | Use cloud/remote storage as overflow; stagger data capture if needed                                  |
| Inability to distinguish real signals from noise or artifacts                      | Calibration incomplete, software filters untested, lack of baseline data                              | Capture calibration/baseline data before event; validate signal processing algorithms                      | Parallel manual review of flagged signals; cross-check with other observatories if possible           |
| Missed signals due to team miscommunication or unclear roles                       | Rushed decision-making, unclear task assignments, information silos                                   | Predefine roles for emergency activation; establish clear communication protocols                          | Daily briefings during event; use checklists for critical steps; assign a “decision captain”          |
| Overlooked safety protocols leading to cascading hardware/software failures         | Pressure to act quickly, skipping checklists, lack of oversight                                       | Mandate safety checklist completion before activation; assign safety officer                               | Require sign-off from multiple leads before critical actions                                          |
| Failure to document changes or decisions, leading to confusion and repeat errors   | Hasty troubleshooting, lack of centralized log, informal communication                                | Use shared digital log for all changes/decisions; assign scribe during crisis                              | Record all interventions in real time; review logs post-event for lessons learned                     |
| Team fatigue and cognitive overload, increasing error rates                        | Extended shifts, high stress, insufficient rest                                                       | Enforce shift rotations; provide mental health support                                                     | Schedule mandatory breaks; bring in backup staff if possible                                          |
| Inadequate contingency plans for unexpected failures                               | Focus on event capture, neglecting “what if” scenarios                                                | Develop rapid response playbooks for common failure modes                                                  | Conduct tabletop exercises; pre-assign troubleshooting teams                                          |
| Amplification of small errors due to time pressure and system interdependencies    | Tight deadlines, lack of system-level thinking, domino effect of minor issues                         | Map system dependencies; stress-test weakest links                                                         | Pause for “go/no-go” checkpoints at each critical phase                                               |


  
Summary of Pre-Mortem Analysis:  
Key insights reveal that rushing to activate the telescope without adequate software testing, calibration, and safety checks can result in data corruption, hardware damage, and missed scientific opportunities. Communication breakdowns, unclear roles, and fatigue compound risks under time pressure. Preventive measures include targeted validation, strict safety protocols, clear team structures, and real-time documentation. Fail-safes such as automated shutdowns, data backups, and contingency planning are essential to balance the urgency of capturing a rare event with the need to safeguard the facility and mission.